05-2
교차 검증과 그리드 서치

In [1]:
import pandas as pd
wine=pd.read_csv('http://bit.ly/wine_csv_data')

In [3]:
# class열을 타깃으로 하고 나머지 열은 특성 배열에 저장
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [4]:
#  훈련세트와 테스트 세트 나누기
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [11]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [9]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [12]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


교차검증

In [13]:
# cross_validate() 교차 검증 함수
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01118016, 0.02397823, 0.01962686, 0.0104835 , 0.01424074]), 'score_time': array([0.00171709, 0.00507331, 0.00164557, 0.00143981, 0.00183296]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [14]:
#  교차 검증의 최종점수 확인 - test_score키에 담긴 5개의 점수를 평균
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


cross_validate() 함수는 기본적으로 회귀 모델일 경우 KFold 분할기를 사용하고,
분류 모델일 경우 StratifiedKFold를 사용.

In [16]:
# StratifiedKFold 사용 교차검증
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [17]:
# 훈련 세트를 섞은 후 10-폴드 교차 검증 수행_n_splits 매개변수는 몇(k) 폴드 교차 검증을 할지 지정
splitter = StratifiedKFold(n_splits=10, shuffle = True, random_state =42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


하이퍼파라미터 튜닝
->모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터

In [19]:
# 그리드 서치사용
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001,0.0002,0.003,0.004,0.0005]}

In [20]:
# 그리드 서치 객체 만들기-n_jobs 매개변수에서 병렬 실행에 사용할 CPU 코어 수 지정. (기본값은 1, -1로 지정하면 시스템에 있는 모든 코어 사용)
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [22]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.003, 0.004,
                                                   0.0005]})

In [24]:
# best_estimator_ -> 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델을 훈련
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [25]:
# 최적의 매개변수는 best_params_에 저장
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [26]:
# cv_results 속성의 'mean_test_score'키 ->각 매개변수에서 수행한 교차 검증의 평균 점수 저장
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.85607111 0.85414803 0.86761605]


In [28]:
#  가장 큰값의 인덱스 추출
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [29]:
# min_impurity_decrease->노드를 분할하기 위한 분순도 감소 최소량 지정.
# max_depth -> 트리의 깊이 제한
# min_samples_split -> 노드를 나누기 위한 최소 샘플 수
#  교차 검증 횟수 = 9x15x10=1,350개  기본5-폴드 교차 검증을 수행하므로 만들어지는 모델의 수는 6,750개.
params = {'min_impurity_decrease':np.arange(0.0001,0.001,0.0001),
          'max_depth': range(5,20,1),
          'min_samples_split': range(2,100,10)}


In [31]:
# n_jobs=-1로 설정하고 그리드 서치 실행.
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [32]:
#  최상의 매개변수 조합 확인
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [33]:
# 최상의 교차 검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


랜덤 서치
->매개변수의 값이 수치일때 값의 범위나 간격을 미리 정하기 어려울수 있다.
또 너무 많은 매개변수 조건이 있어 그리드 서치 수행 시간이 오래 걸릴 수 있을 때 사용.

In [34]:
# 싸이파이
from scipy.stats import uniform, randint

In [35]:
# 10개의 숫자 샘플링
rgen = randint(0,10)
rgen.rvs(10)

array([2, 7, 4, 6, 6, 6, 8, 9, 1, 0])

In [36]:
# 1,000개 샘플링
np.unique(rgen.rvs(1000), return_counts= True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([107, 101, 111, 103,  86,  98, 106,  92,  83, 113]))

In [39]:
# 0~1 사이의 실수 추출
ugen = uniform(0,1)
ugen.rvs(10)

array([0.03881392, 0.41130745, 0.91783332, 0.66678774, 0.03273645,
       0.75699692, 0.98218169, 0.65060771, 0.96540712, 0.85313187])

In [41]:
params = {'min_impurity_decrease':uniform(0.0001,0.001),
          'max_depth': randint(20,50),
          'min_samples_split': randint(2,25),
          'min_samples_leaf': randint(1,25)}

In [42]:
from sklearn.model_selection import RandomizedSearchCV
gs=RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                      n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x78bf34b95810>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x78bf34b951b0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x78bf34b96620>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x78bf34b977f0>},
                   random_state=42)

In [43]:
#  최적의 매개변수 조합 출력
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [44]:
#  최고의 교차 검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [45]:
# 최종 모델로 결정하고 테스트 세트의 성능 확인
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86


확인 문제 3.
마지막 RandomizedSearchCV 예제에서 DecisionTreeClassifier 클래스에 splitter='random' 매개변수를 추가하고 다시 훈련해 보세요.
splitter 매개변수의 기본값은 'best'로 각 노드에서 최선의 분할을 찾습니다.
'random' 이면 무작위로 분할한 다음 가장 좋은것을 고릅니다.

In [46]:
from sklearn.model_selection import RandomizedSearchCV
gs=RandomizedSearchCV(DecisionTreeClassifier(random_state=42, splitter='random'), params,
                      n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x78bf34b95810>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x78bf34b951b0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x78bf34b96620>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x78bf34b977f0>},
                   random_state=42)

In [47]:
print(gs.best_params_)

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}


In [48]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8458726956392981


In [49]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.786923076923077
